# Lesson 6.1: Customizing Chains and Prompts with LCEL

---

In previous Modules, we became familiar with **Chains** as a way to connect LangChain components in a fixed processing flow. However, when building more complex applications, you'll need the ability to customize and combine components more flexibly. This is where **LangChain Expression Language (LCEL)** comes into play.

This lesson will delve into LCEL, how to build complex and flexible chains, and how to customize the behavior of components within the chain.

## 1. Diving Deeper into LangChain Expression Language (LCEL)

### 1.1. What is LCEL?

**LangChain Expression Language (LCEL)** is a declarative language that allows you to easily connect LangChain components (such as Prompts, LLMs, Parsers, Retrievers, and custom functions) into processing chains that can pass data back and forth.

* **Declarative:** You describe how components should be connected, rather than writing sequential execution steps.
* **Composable:** Components built with LCEL can be easily combined using the pipe (`|`) operator.
* **Streamable:** LCEL supports streaming output, allowing applications to respond faster.
* **Debuggable:** Easy to trace data flow and intermediate steps.
* **Runnable in parallel:** Some parts of the chain can run in parallel to optimize performance.



### 1.2. Why Use LCEL?

* **Flexibility:** Build more complex processing flows than traditional `SequentialChain`.
* **Modularity:** Each component is an independent `Runnable`, easy to reuse and test.
* **Performance:** Supports asynchronous (async) and parallel execution, improving speed.
* **Extensibility:** Easily add custom processing steps or integrate external systems.
* **Transparency:** The declarative structure makes it easy to read and understand the data flow.


---

## 2. Building Complex and Flexible Chains with Runnable

In LCEL, every component is a `Runnable`. A `Runnable` is a standard interface that allows components to be invoked, streamed, or run asynchronously (`ainvoke`, `astream`).

You can combine `Runnable`s using the pipe (`|`) operator to form processing chains.

### 2.1. `RunnableSequence` (Sequential Connection)

* **Concept:** Connects `Runnable`s in sequence, where the output of one is the input of the next. This is the most common way to build a chain.
* **Usage:** Use the pipe (`|`) operator.

In [ ]:
# Cài đặt thư viện nếu chưa có
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Thiết lập biến môi trường cho khóa API của OpenAI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Định nghĩa Prompt
prompt = ChatPromptTemplate.from_template("Kể một câu chuyện ngắn về {topic} trong khoảng 50 từ.") # Tell a short story about {topic} in about 50 words.

# Định nghĩa Parser
parser = StrOutputParser()

# Xây dựng chuỗi tuần tự với LCEL
# Đầu vào của chuỗi là 'topic'
# Prompt nhận 'topic' và tạo tin nhắn
# LLM nhận tin nhắn và tạo phản hồi
# Parser nhận phản hồi và chuyển thành chuỗi
story_chain = prompt | llm | parser

print("--- Ví dụ RunnableSequence ---") # Example RunnableSequence
print(story_chain.invoke({"topic": "một con mèo phiêu lưu"})) # an adventurous cat
print("-" * 30)

### 2.2. `RunnableParallel` (Run in Parallel)

* **Concept:** Allows you to run multiple `Runnable`s simultaneously and combine their results into a dictionary. Very useful when you need to prepare multiple inputs for a subsequent step.
* **Usage:** Use a dictionary literal.

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Chuỗi để tạo câu chuyện
story_chain = ChatPromptTemplate.from_template("Kể một câu chuyện ngắn về {topic} trong khoảng 50 từ.") | llm | StrOutputParser() # Tell a short story about {topic} in about 50 words.

# Chuỗi để tạo một câu thơ
poem_chain = ChatPromptTemplate.from_template("Viết một bài thơ ngắn về {topic} trong khoảng 4 dòng.") | llm | StrOutputParser() # Write a short poem about {topic} in about 4 lines.

# Kết hợp hai chuỗi trên để chạy song song
# Đầu vào của parallel_chain là 'topic'
# 'story' sẽ chạy story_chain với 'topic'
# 'poem' sẽ chạy poem_chain với 'topic'
parallel_chain = RunnableParallel(
    story=story_chain,
    poem=poem_chain
)

print("--- Ví dụ RunnableParallel ---") # Example RunnableParallel
result = parallel_chain.invoke({"topic": "mùa thu"}) # autumn
print("Câu chuyện:\n", result["story"]) # Story:
print("\nBài thơ:\n", result["poem"]) # Poem:
print("-" * 30)

### 2.3. `RunnablePassthrough` (Pass Input Through)

* **Concept:** A simple `Runnable` that passes its input directly as its output. Very useful when you need to pass a portion of the original input to a later step in the chain, or when you need to combine inputs from multiple sources.
* **Usage:** Often used within `RunnableParallel` to retain an input variable.

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Ví dụ: Truyền câu hỏi của người dùng qua đồng thời với kết quả tìm kiếm
# Giả sử có một retriever (từ Module 3)
# from langchain_community.vectorstores import Chroma
# from langchain_openai import OpenAIEmbeddings
# from langchain_core.documents import Document
# embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
# vector_store = Chroma.from_documents([Document(page_content="LangChain là một framework để xây dựng ứng dụng LLM."), Document(page_content="RAG giúp giảm ảo giác.")], embeddings)
# retriever = vector_store.as_retriever()

# Để ví dụ đơn giản, chúng ta sẽ giả lập retriever
def mock_retriever(query):
    if "LangChain" in query:
        return "LangChain là một framework để xây dựng ứng dụng LLM." # LangChain is a framework for building LLM applications.
    if "RAG" in query:
        return "RAG giúp giảm ảo giác và tăng độ chính xác." # RAG helps reduce hallucinations and increase accuracy.
    return "Không tìm thấy thông tin liên quan." # No relevant information found.

# Định nghĩa Prompt cho RAG
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Trả lời câu hỏi dựa trên ngữ cảnh: {context}"), # Answer the question based on the context:
    ("human", "{question}"),
])

# Xây dựng chuỗi RAG với RunnablePassthrough
# Đầu vào của chuỗi là {"question": "..."}
rag_chain_with_passthrough = (
    {
        "context": RunnableLambda(mock_retriever), # Retriever receives question from input
        "question": RunnablePassthrough() # Pass the original question through
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("--- Ví dụ RunnablePassthrough ---") # Example RunnablePassthrough
print(rag_chain_with_passthrough.invoke({"question": "LangChain là gì?"})) # What is LangChain?
print("-" * 30)

### 2.4. Customizing Component Behavior in the Chain

LCEL allows you to customize the behavior of individual components or the entire chain:

* **`.bind()`**: Attaches fixed parameters to a `Runnable`. Example: `llm.bind(stop=["\nObservation"])`.
* **`.with_config()`**: Configures runtime settings like `callbacks`, `tags`, `metadata`.
* **`.with_retry()`**: Adds retry logic for a `Runnable` if it fails.
* **`.with_types()`**: Adds type information for inputs/outputs, useful for validation and auto-completion.
* **`.map()`**: Applies a `Runnable` to each element in a list input.

In [ ]:
from langchain_core.runnables import RunnableLambda

# Ví dụ về .bind() và .with_config()
# Giả sử bạn muốn một LLM luôn dừng lại khi thấy "END"
bound_llm = llm.bind(stop=["END"]) # Example of .bind() and .with_config() / Assume you want an LLM to always stop when it sees "END"

# Một chuỗi đơn giản với LLM đã bind
bound_chain = ChatPromptTemplate.from_template("Viết một đoạn văn ngắn về {topic}. Kết thúc bằng từ END.") | bound_llm | StrOutputParser() # A simple chain with the bound LLM / Write a short paragraph about {topic}. End with the word END.

print("--- Ví dụ .bind() ---") # Example .bind()
print(bound_chain.invoke({"topic": "mùa đông"})) # winter
print("-" * 30)

# Ví dụ về .with_config()
# Thêm metadata cho chuỗi để theo dõi
configured_chain = (
    ChatPromptTemplate.from_template("Nói xin chào.") # Say hello.
    | llm
    | StrOutputParser()
).with_config(tags=["greeting_chain", "test_run"]) # Add metadata to the chain for tracking

print("--- Ví dụ .with_config() ---") # Example .with_config()
print(configured_chain.invoke({})) # No input needed as prompt is fixed
print("-" * 30)


---

## 3. Practical Example: Building a Custom RAG Chain with LCEL

We will build a complete and custom RAG chain using LCEL, combining components learned from Module 3 and Module 5 (Memory).

**Preparation:**
* Ensure you have the necessary libraries installed: `langchain-openai`, `chromadb`, `pypdf`.
* Set the `OPENAI_API_KEY` environment variable.
* Create a sample PDF file (e.g., `lcel_rag_document.pdf`).

In [ ]:
# Cài đặt thư viện nếu chưa có
# pip install langchain-openai openai chromadb pypdf

import os
import shutil
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory # Để duy trì lịch sử trò chuyện

# Thiết lập biến môi trường cho khóa API của OpenAI
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Khởi tạo LLM và Embeddings Model
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# Thư mục lưu trữ ChromaDB
persist_directory = "./chroma_lcel_rag_demo_db"
# Xóa thư mục cũ nếu tồn tại để đảm bảo sạch sẽ
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
    print(f"Đã xóa thư mục Chroma cũ: {persist_directory}") # Old Chroma directory deleted:

# Tạo một file PDF mẫu
pdf_file_path = "lcel_rag_document.pdf"
try:
    from reportlab.pdfgen import canvas
    c = canvas.Canvas(pdf_file_path)
    c.drawString(100, 750, "LangChain Expression Language (LCEL) là một cách mạnh mẽ để xây dựng các chuỗi.") # LangChain Expression Language (LCEL) is a powerful way to build chains.
    c.drawString(100, 730, "LCEL cho phép kết hợp các Runnable bằng toán tử pipe (|).") # LCEL allows combining Runnables using the pipe operator (|).
    c.drawString(100, 710, "Các loại Runnable bao gồm RunnableSequence, RunnableParallel và RunnablePassthrough.") # Runnable types include RunnableSequence, RunnableParallel, and RunnablePassthrough.
    c.drawString(100, 690, "Bạn có thể tùy chỉnh Runnable bằng .bind(), .with_config(), v.v.") # You can customize Runnables using .bind(), .with_config(), etc.
    c.drawString(100, 670, "RAG (Retrieval-Augmented Generation) kết hợp truy xuất thông tin với LLM.") # RAG (Retrieval-Augmented Generation) combines information retrieval with LLM.
    c.drawString(100, 650, "Chroma là một Vector Store phổ biến cho việc lưu trữ embeddings.") # Chroma is a popular Vector Store for storing embeddings.
    c.save()
    print(f"Đã tạo file PDF mẫu: {pdf_file_path}") # Sample PDF file created:
except ImportError:
    with open(pdf_file_path, "w") as f:
        f.write("Đây là file giả lập PDF cho LCEL RAG. Vui lòng thay bằng PDF thật.") # This is a dummy PDF file for LCEL RAG. Please replace with a real PDF.
    print("Không thể tạo PDF thật bằng reportlab. Sử dụng file giả lập.") # Could not create real PDF with reportlab. Using dummy file.
    print("Vui lòng đảm bảo bạn có file PDF thật 'lcel_rag_document.pdf' để ví dụ hoạt động tốt nhất.") # Please ensure you have a real PDF file 'lcel_rag_document.pdf' for the example to work best.

# --- Giai đoạn Indexing (Tải, Chia nhỏ, Nhúng, Lưu trữ) ---
print("\n--- Bắt đầu giai đoạn Indexing ---") # --- Starting Indexing phase ---
loader = PyPDFLoader(pdf_file_path)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)
vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory)
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
print("Đã hoàn thành giai đoạn Indexing và tạo Retriever.") # Indexing phase completed and Retriever created.

# --- Giai đoạn Runtime (Xây dựng chuỗi RAG tùy chỉnh với LCEL và Memory) ---

# 1. Khởi tạo Memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 2. Định nghĩa Prompt cho RAG
# Prompt này sẽ bao gồm lịch sử trò chuyện, ngữ cảnh và câu hỏi hiện tại
rag_prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", "Bạn là một trợ lý Q&A hữu ích. Trả lời câu hỏi dựa trên ngữ cảnh được cung cấp. Nếu không tìm thấy, hãy nói không biết. Duy trì ngữ cảnh cuộc trò chuyện."), # You are a helpful Q&A assistant. Answer the question based on the provided context. If not found, say you don't know. Maintain conversation context.
    MessagesPlaceholder(variable_name="chat_history"), # Lịch sử trò chuyện
    ("human", "Ngữ cảnh: {context}\n\nCâu hỏi: {question}"), # Context: {context}\n\nQuestion: {question}
])

# 3. Xây dựng chuỗi RAG tùy chỉnh với LCEL
# Chuỗi này sẽ:
# - Lấy câu hỏi của người dùng (input)
# - Lấy lịch sử trò chuyện từ memory
# - Truy xuất ngữ cảnh từ retriever
# - Kết hợp lịch sử, ngữ cảnh và câu hỏi vào prompt
# - Gọi LLM để tạo câu trả lời
# - Lưu ngữ cảnh mới vào memory
# - Trả về câu trả lời

# Hàm để định dạng tài liệu được truy xuất thành một chuỗi
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chuỗi chính
# Đầu vào của chuỗi là {'input': user_question}
# RunnableParallel để lấy cả lịch sử và ngữ cảnh song song
# Sau đó, kết hợp chúng vào prompt và gọi LLM

# Chuỗi để lấy ngữ cảnh và lịch sử
context_and_history = RunnableParallel(
    context=RunnableLambda(lambda x: x["input"]) | retriever | format_docs, # Truy xuất ngữ cảnh
    chat_history=RunnableLambda(lambda x: memory.load_memory_variables({})["chat_history"]), # Lấy lịch sử từ memory
    question=RunnablePassthrough() # Truyền câu hỏi gốc qua
)

# Chuỗi RAG cuối cùng
full_rag_chain = (
    context_and_history
    | rag_prompt_with_history
    | llm
    | StrOutputParser()
)

# Hàm để gọi chuỗi và lưu ngữ cảnh vào memory
def invoke_and_save(chain, user_input):
    print(f"\nNgười dùng: {user_input}") # User:
    # Lấy lịch sử hiện tại trước khi invoke
    current_chat_history = memory.load_memory_variables({})["chat_history"]
    
    # Invoke chuỗi với input và lịch sử
    response = chain.invoke({"input": user_input, "chat_history": current_chat_history})
    
    # Lưu ngữ cảnh mới vào memory
    memory.save_context(
        {"input": user_input},
        {"output": response}
    )
    print(f"AI: {response}")
    return response

# --- Thực thi hệ thống Q&A tùy chỉnh ---
print("\n--- Bắt đầu hệ thống Q&A RAG tùy chỉnh với LCEL và Memory ---") # Starting custom RAG Q&A system with LCEL and Memory

# Câu hỏi 1: Về LCEL
invoke_and_save(full_rag_chain, "LCEL là gì?") # What is LCEL?

# Câu hỏi 2: Về RAG (sẽ sử dụng ngữ cảnh từ PDF)
invoke_and_save(full_rag_chain, "RAG giúp ích gì?") # What does RAG help with?

# Câu hỏi 3: Hỏi về một khái niệm đã nói trước đó (kiểm tra memory)
invoke_and_save(full_rag_chain, "Nó có những thành phần nào?") # What are its components?

# Câu hỏi 4: Hỏi về một khái niệm không có trong tài liệu (LLM sẽ nói không biết)
invoke_and_save(full_rag_chain, "Thủ đô của Úc là gì?") # What is the capital of Australia?

print("\n--- Lịch sử cuối cùng trong Memory ---") # Final history in Memory
print(memory.load_memory_variables({}))

# Dọn dẹp file PDF mẫu và thư mục Chroma
os.remove(pdf_file_path)
if os.path.exists(persist_directory):
    shutil.rmtree(persist_directory)
print(f"\nĐã xóa file PDF mẫu và thư mục Chroma '{persist_directory}'.") # Sample PDF file and Chroma directory '{persist_directory}' deleted.

print("\n--- Kết thúc hệ thống Q&A RAG tùy chỉnh ---") # Ending custom RAG Q&A system


**Explanation of data flow in `full_rag_chain`:**

1.  `invoke_and_save(full_rag_chain, user_input)`: This function takes the user's question and calls `full_rag_chain`.
2.  `context_and_history`:
    * `context=RunnableLambda(lambda x: x["input"]) | retriever | format_docs`: This part takes the `input` (user's question) from the input dictionary, passes it to the `retriever` to retrieve documents, and then formats those documents into a string. The result is assigned to the `context` key.
    * `chat_history=RunnableLambda(lambda x: memory.load_memory_variables({})["chat_history"])`: This part retrieves the current conversation history from the initialized `memory` object. The result is assigned to the `chat_history` key.
    * `question=RunnablePassthrough()`: This part passes the original user's question directly to the `question` key.
    * The output of `context_and_history` is a dictionary: `{"context": "...", "chat_history": [...], "question": "..."}`.
3.  `| rag_prompt_with_history`: The prompt receives this dictionary and uses the `context`, `chat_history`, and `question` keys to construct the final prompt sent to the LLM.
4.  `| llm`: The LLM receives the constructed prompt and generates a response.
5.  `| StrOutputParser()`: The parser converts the LLM's response into a text string.
6.  `memory.save_context(...)`: After the LLM generates the response, the `invoke_and_save` function saves this (input, output) pair to `memory` to maintain context for subsequent turns.


---

## Lesson Summary

This lesson delved into **LangChain Expression Language (LCEL)**, a powerful tool for building complex and flexible processing chains in LangChain. You learned how to combine `Runnable`s like `RunnableSequence`, `RunnableParallel`, and `RunnablePassthrough` to create custom data flows. We also explored how to customize the behavior of components using methods like `.bind()` and `.with_config()`. Finally, you applied all this knowledge to **practice building a custom RAG chain with LCEL and integrated Memory**, illustrating how different LangChain components can be seamlessly connected to create intelligent and context-aware LLM applications.